## Ансамбли и полносвязные нейронные сети
В этом ноутбуке вам нужно обучить модели на датасете классификации из предыдущего ноутбука и сравнить результаты. Вам будет предоставлен baseline, на основе которого вы будете доделывать предсказывающие модели. Оценка лабы будет зависеть от ROC-AUC на тестовых данных по следующим критериям:
\
AUC - на тестовых данных
- $AUC \leq 0.76$ - 0 баллов
- $0.76 < AUC \leq 0.77$ - 2 балла
- $0.77 < AUC \leq 0.78$ - 4 балла
- $0.78 < AUC \leq 0.79$ - 6 баллов
- $0.79 < AUC \leq 0.80$ - 8 баллов
- $AUC > 0.80$ - 10 баллов


In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# Загрузка данных
data = pd.read_csv('german.csv', sep=';')  # Загрузка датасета из CSV файла
print(data.head())  # Вывод первых нескольких строк датасета для предварительного просмотра

# Определение признаков и целевой переменной
X = data.iloc[:, 1:].to_numpy()  # Все столбцы, кроме первого, используются как признаки
y = data.iloc[:, 0].to_numpy()  # Первый столбец используется как целевая переменная

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Данные разделяются на 80% для обучения и 20% для тестирования, с фиксированным случайным состоянием для воспроизводимости

In [ ]:
# Визуализация распределения классов
plt.hist(y_train, bins=2, edgecolor='k')  # Гистограмма для визуализации распределения классов
plt.xticks([0, 1])  # Установка меток по оси X
plt.xlabel('Class (0: Non-Creditworthy, 1: Creditworthy)')  # Подпись оси X
plt.ylabel('Count')  # Подпись оси Y
plt.title('Distribution of Classes in Training Data')  # Заголовок графика
plt.show()  # Показ графика

In [ ]:
# Определение функции для оценки моделей
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)  # Обучение модели на обучающей выборке
    y_pred = model.predict(X_test)  # Прогнозирование на тестовой выборке
    
    # Вычисление различных метрик
    roc_auc = roc_auc_score(y_test, y_pred)  # ROC AUC
    accuracy = accuracy_score(y_test, y_pred)  # Точность
    precision = precision_score(y_test, y_pred)  # Полнота
    recall = recall_score(y_test, y_pred)  # Признак
    
    return roc_auc, accuracy, precision, recall  # Возврат всех метрик

# Baseline модели
baseline_rf = RandomForestClassifier(n_estimators=100, random_state=42)  # Базовая модель Random Forest
baseline_gb = GradientBoostingClassifier(n_estimators=100, random_state=42)  # Базовая модель Gradient Boosting
baseline_mlp = MLPClassifier(hidden_layer_sizes=(30,), max_iter=500, random_state=42)  # Базовая модель MLP

# Оценка базовых моделей
models = {
    "Random Forest": baseline_rf,
    "Gradient Boosting": baseline_gb,
    "MLP": baseline_mlp
}

results = {}  # Словарь для хранения результатов

# Цикл по всем моделям для их оценки
for model_name, model in models.items():
    roc_auc, accuracy, precision, recall = evaluate_model(model, X_train, y_train, X_test, y_test)  # Оценка модели
    results[model_name] = {  # Сохранение результатов
        "ROC AUC": roc_auc,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall
    }

# Вывод результатов базовых моделей
for model_name, metrics in results.items():
    print(f"{model_name} метрики:")  # Заголовок для каждой модели
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.2f}")  # Печать метрик с округлением до двух знаков
    print()  # Пустая строка для разделения выводов


In [ ]:
# Оптимизация моделей с помощью GridSearchCV
param_grid_rf = {  # Параметры для Random Forest
    'n_estimators': [100, 200],  # Количество деревьев
    'max_depth': [None, 10, 20],  # Максимальная глубина деревьев
    'min_samples_split': [2, 5]  # Минимальное количество образцов для разделения
}

param_grid_gb = {  # Параметры для Gradient Boosting
    'n_estimators': [100, 200],  # Количество деревьев
    'learning_rate': [0.01, 0.1],  # Скорость обучения
    'max_depth': [3, 5, 7]  # Максимальная глубина деревьев
}

param_grid_mlp = {  # Параметры для MLP
    'hidden_layer_sizes': [(30,), (50,), (100,)],  # Размеры скрытых слоев
    'max_iter': [500, 1000]  # Максимальное количество итераций
}

# Поиск лучших параметров для Random Forest
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, scoring='roc_auc')  # Инициализация GridSearchCV
grid_rf.fit(X_train, y_train)  # Обучение с использованием кросс-валидации
best_rf = grid_rf.best_estimator_  # Получение лучшей модели

# Поиск лучших параметров для Gradient Boosting
grid_gb = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid_gb, cv=5, scoring='roc_auc')  # Инициализация GridSearchCV
grid_gb.fit(X_train, y_train)  # Обучение
best_gb = grid_gb.best_estimator_  # Получение лучшей модели

# Поиск лучших параметров для MLP
grid_mlp = GridSearchCV(MLPClassifier(random_state=42), param_grid_mlp, cv=5, scoring='roc_auc')  # Инициализация GridSearchCV
grid_mlp.fit(X_train, y_train)  # Обучение
best_mlp = grid_mlp.best_estimator_  # Получение лучшей модели

# Оценка оптимизированных моделей
optimized_models = {
    "Optimized Random Forest": best_rf,
    "Optimized Gradient Boosting": best_gb,
    "Optimized MLP": best_mlp
}

optimized_results = {}  # Словарь для хранения результатов оптимизированных моделей

# Цикл по оптимизированным моделям для их оценки
for model_name, model in optimized_models.items():
    roc_auc, accuracy, precision, recall = evaluate_model(model, X_train, y_train, X_test, y_test)  # Оценка модели
    optimized_results[model_name] = {  # Сохранение результатов
        "ROC AUC": roc_auc,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall
    }

# Вывод результатов оптимизированных моделей
for model_name, metrics in optimized_results.items():
    print(f"{model_name} метрики:")  # Заголовок для каждой оптимизированной модели
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value:.2f}")  # Печать метрик с округлением до двух знаков
    print()  # Пустая строка для разделения выводов

## Экспериментируйте
Для получения лучшего качества придется поэкспериментировать. Подсказка: попробуйте оптимизировать гиперпараметры модели